In [2]:
import tensorflow as tf

### 持久化

#### 1. 保存计算两个变量和的模型。

In [3]:
v1 = tf.Variable(tf.random_normal([1], stddev=1, seed=1))
v2 = tf.Variable(tf.random_normal([1], stddev=1, seed=1))
result = v1 + v2

init_op = tf.global_variables_initializer()
saver = tf.train.Saver()

In [4]:
with tf.Session() as sess:
    sess.run(init_op)
    print(sess.run(v1))
    print(sess.run(v2))
    saver.save(sess, "saved_model/model.ckpt")

[-0.8113182]
[-0.8113182]


#### 2. 加载保存了两个变量和的模型。

In [7]:
with tf.Session() as sess:
    saver.restore(sess, "saved_model/model.ckpt")
    print(sess.run(result))

INFO:tensorflow:Restoring parameters from saved_model/model.ckpt
[-1.6226364]


#### 3. 直接加载持久化的图。因为之前没有导出v3，所以这里会报v3未初始化错误。

In [10]:
v3 = tf.Variable(tf.random_normal([1], stddev=1, seed=1))
saver = tf.train.import_meta_graph("saved_model/model.ckpt.meta")

with tf.Session() as sess:
    saver.restore(sess, "saved_model/model.ckpt")
    print(sess.run(v1))
    print(sess.run(v2))
    print(sess.run(v3))

INFO:tensorflow:Restoring parameters from saved_model/model.ckpt
[-0.8113182]
[-0.8113182]


FailedPreconditionError: Attempting to use uninitialized value Variable_8
	 [[Node: Variable_8/_16 = _Send[T=DT_FLOAT, client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_4_Variable_8", _device="/job:localhost/replica:0/task:0/device:GPU:0"](Variable_8)]]

#### 4. 变量重命名。

In [11]:
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name = "other-v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name = "other-v2")
saver = tf.train.Saver({"v1": v1, "v2": v2})

### 持久化原理

#### 1. json格式导出MetaGraphDef Protoacl Buffer

In [5]:
saver.export_meta_graph("saved_model/model.ckpt.meta.json", as_text=True)

meta_info_def {
  stripped_op_list {
    op {
      name: "Add"
      input_arg {
        name: "x"
        type_attr: "T"
      }
      input_arg {
        name: "y"
        type_attr: "T"
      }
      output_arg {
        name: "z"
        type_attr: "T"
      }
      attr {
        name: "T"
        type: "type"
        allowed_values {
          list {
            type: DT_HALF
            type: DT_BFLOAT16
            type: DT_FLOAT
            type: DT_DOUBLE
            type: DT_UINT8
            type: DT_INT8
            type: DT_INT16
            type: DT_INT32
            type: DT_INT64
            type: DT_COMPLEX64
            type: DT_COMPLEX128
            type: DT_STRING
          }
        }
      }
    }
    op {
      name: "Assign"
      input_arg {
        name: "ref"
        type_attr: "T"
        is_ref: true
      }
      input_arg {
        name: "value"
        type_attr: "T"
      }
      output_arg {
        name: "output_ref"
        type_attr: "T"
        